# TP6 - corps binaires et applications

<span class='alert-info'> Répondez aux questions dans un notebook séparé que vous rendrez au format <strong> NOM1_NOM2_tp6.ipynb </strong> </span>

## Partie 1 - un anneau qui a du caractère

On va travailler dans un **anneau** (structure algébrique où $+, -, \times$ sont autorisées) **fini** à $64$ éléments noté $(\mathbb{F}_{64}, +, \times)$. Les éléments de $\mathbb{F}_{64}$ seront informatiquement des instances de la classe ```F``` implémentée ci-dessous. 

In [ ]:
class F(SageObject):
    
    # names for the 64 elements
    # static variables 
    names = "01abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ !?-'#$%&@"
    
    # create F object associated to character val 
    def __init__(self, val):
        
        try:
            # id = integer to identify the ring element 
            self.id = Integer(F.names.index(str(val)))
            
        except ValueError:
            # val is not found in names 
            raise ValueError("Element %s unknown" % val)
            
    # F1 == F2 if F1 and F2 have the same id 
    def __eq__(self, other):
        
        return self.id == other.id
        
    def __ne__(self, other):
        
        return not (self == other)
       
    # special method to represent an instance of F 
    def __repr__(self):
        
        return F.names[self.id]
    
    # resultat of F1 + F2
    def __add__(self, other):
        
        r = F(self)
        r += other
        return r
    
    # result of F1 += F2
    def __iadd__(self, other):
        
        # add two objects = bitwise addition on their id 
        self.id = self.id.__xor__(other.id)
        return self
    
    # method that is used to perform F1 * F2 
    def shift(self):
        
        # append a 0 in the binary representation of self.id 
        # tmp is an integer coded with 7 bits 
        tmp = self.id << 1
        
        # res stores bits associated to 2^6 to 2^0 
        res = tmp % 64
        
        # tmp // 64 stores the bit associated to 2^7  
        # if the bit is 1...
        if tmp // 64:
            
            # 1 000 000 is represented by 0 011 011 (why?)
            # perform bitwise addition 011 011 + res  
            res = res.__xor__(27) 
            
        self.id = res
    
    # result of F1 * F2 
    # magic multiplication 
    def __mul__(self, other):
        
        res = F(0)
        tmp = F(other)
        
        # F1 is represented as a binary word 
        # when a bit is 0, do nothing 
        # when a bit is 1, add a shifted version of F(other)
        for b in self.id.bits():
            if b:
                res += tmp
            tmp.shift()
            
        return res
        
    # resultat of F*F*...*F
    def __pow__(self, exp):
        
        res = F(1)
        
        for i in range(exp):
            res *= self
        
        return res
        
F.elements = [F(i) for i in F.names]

Les éléments de $\mathbb{F}_{64}$ sont les 64 caractères (ou *symboles*) suivants:

In [ ]:
print(F.elements)

On crée facilement les *objets* associés à chaque élément. Par exemple, pour créer l'objet associé à l'élément $a$: 

In [ ]:
a = F("a")
a

Deux opérations de base sont implémentées : une **addition**

In [ ]:
c = F("c")
a + c

et une **multiplication**

In [ ]:
a * c

Notez que l'anneau $(\mathbb{F}_{64}, +, \times)$ n'est **pas** isomorphe à $(\mathbb{Z}/64 \mathbb{Z}, +, \times)$. On peut par exemple vérifier que $\alpha + \alpha = 0$ pour tout $\alpha \in \mathbb{F}_{64}$ (alors que dans $\mathbb{Z}/64\mathbb{Z}$, chaque élément n'est pas son propre opposé) 

In [ ]:
alpha = choice(F.elements)  # random element, run again for another one

print("alpha = %s, alpha + alpha = %s" % (alpha, alpha + alpha))

### a) C'est un anneau commutatif

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

Vérifier explicitement (par *force brute*) que **tous** les axiomes d'anneau commutatif sont satisfaits pour cette structure algébrique.

### b) Inverse d'un élément

Il est aisé de vérifier qu'un élément est inversible si on nous fournit un candidat pour son inverse. Par exemple: le calcul ci-dessous suffit à convaincre que l'élément $\mathtt{a}$ est inversible et que $\mathtt{a}^{-1} = \mathtt{R}$.

In [ ]:
F("a") * F("R")

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

Soit $m$ la somme des numéros des mois de naissance des membres du binôme et $\zeta$ l'élément `F.elements[m]` de $\mathbb{F}_{64}$. Vérifiez que $\zeta$ est inversible en recherchant par force brute son inverse parmi les éléments de $\mathbb{F}_{64}$.

### c) Structure multiplicative

Vous l'aurez peut-être deviné : tous les éléments non nuls de $\mathbf{F}$ sont inversibles (il s'agit donc d'un *corps fini* à 64 éléments). 

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

- Déterminer l'ordre multiplicatif de l'élément $\mathtt{a}$
- En déduire que tous les éléments de $\mathbb{F}_{64}$ (sauf le neutre additif) sont inversibles. 
- En partant de $\mathtt{a}^{-1} = \mathtt{R}$, déterminer astucieusement (donc pas par force brute) l'inverse de $\zeta$

### d) Solutions d'une équation polynomiale

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

- Déterminer les racines dans $\mathbb{F}_{64}$ du polynôme $X^6 + X^4 + X^3 + X + 1 \in \mathbb{F}_2[X]$
- Expliquer comment cela permet de se convaincre que chaque élément de $\mathbb{F}_{64}$ peut s'écrire, de façon unique, sous la forme d'une expression de la forme

    $$ \boxed{b_5 \mathtt{a}^5 + b_4 \mathtt{a}^4 + b_3 \mathtt{a}^3 + b_2 \mathtt{a}^2 + b_1 \mathtt{a} + b_0} $$

    avec $(b_0, \ldots, b_5) \in \mathbb{F}_2^6$
- Quelle est l'écriture correspondante pour votre élément $\zeta$ ci-dessus ?

## Partie 2 - chaînes de caractères et applications

Les éléments de $\mathbb{F}_{64}$ étant des caractères, on considérer une chaîne de caractères de longueur $n$ comme un élément de $\mathbb{F}_{64}^n$. Voici un début de classe permettant de considérer une chaîne de caractères (admissibles) comme une liste d'élémentsde $\mathbb{F}_{64}$.

In [ ]:
class V(SageObject):
    
    # constructor 
    # s is a string of eligible characters 
    def __init__(self, s):
        
        self.coeffs = [ F(c) for c in s ]
        
    # representation of a V object 
    def __repr__(self):
        
        return "".join([c.__repr__() for c in self.coeffs])
    
    # result of V1 == V2 
    def __eq__(self, other):
        
        return self.coeffs == other.coeffs
   
    # length of s
    def __len__(self):
        
        return len(self.coeffs)

### a) Somme de vecteurs

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

Ajouter à la classe `V` une méthode `__add__` permettant de faire la somme **terme à terme** de deux chaînes de caractères de même longueur. Par exemple, on devrait avoir:
- `V("ceci") + V("cela") == V("00hg")`
- `V("CECI") + V("cela") == V("yKrK")` 
- `V("ceci") + V("CELA") == V("yKHu")`
- `V("CECI") + V("CELA") == V("00-?")` 

### b) Chiffrement à clé secrète

On peut utiliser l'addition de chaînes de caractères ainsi définie pour garantir la confidentialité d'une chaîne de caractères donnée. Par exemple : soit le message clair `m` ci-dessous. 

In [ ]:
m = V("Texte clair")

On peut le rendre inintelligible par un tiers en le masquant à l'aide d'une clé secrète `k` générée aléatoirement: 

In [ ]:
k = V("kqaS%LAKVw'")

On obtient alors la chaîne incompréhensible suivante calculant la *somme* `c` de `m` et `k`: 

In [ ]:
c = V("Hsz##pwPXqN")

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

En utilisant les propriétés de $\mathbb{F}_{64}$: 
- indiquer comment déchiffrer le code si la clé est connu 
- déchiffrer le message `V("Kcgw&YCVFSl")` codé avec la même clé 

### c) Détection d'erreurs

On peut utiliser la structure algébrique mise sur les caractères pour *détecter* automatiquement des erreurs de cohérence dans une chaîne de caractères (erreur de transmission, media de stockage corrompu, ...). L'idée est toute simple: on ajoute une **somme de contrôle**.  

La version la plus simple consiste à fournir pour chaque chaîne de caractères la somme de tous ceux-ci.

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

Implémenter une fonction pour calculer la somme de contrôle d'un élément de $\mathbb{F}_{64}^n$. Déterminer la somme de contrôle de la phrase `It's burger time!` 

<div class='alert-danger'> <strong> Alerte à la question! </strong> </div>

On peut détecter des erreurs si la somme de contrôle calculée ne correspond pas à la somme de contrôle attendue. Comprenez-le avec l'exemple ci-dessous. 

In [ ]:
v = V("Ici y a-t-il une erreur ? C#mment le savez-vous ?")
s = F("Z") # expected checksum 

<div class='alert-danger'> <strong> Alerte à la culture! </strong> </div>

Notez que si, en plus de détecter la présence d'une erreur dans la chaîne, on sait à quel caractère elle s'est produite, on peut même la corriger automatiquement en lui ajoutant le scalaire approprié ; c'est l'idée de base des codes correcteurs d'erreurs (comme le [code de Reed-Solomon](https://en.wikipedia.org/wiki/Reed%E2%80%93Solomon_error_correction)) qui sont extensivement utilisés dans tous les systèmes de communication modernes.